# Análisis y generación de consultas SQL con modelos de lenguaje

Este notebook muestra el proceso de:
1. Conexión a la base de datos.
2. Carga y preprocesamiento de un *dataset*.
3. Creación de prompts y generación de consultas con diferentes modelos (Deepseek Coder en sus distintas versiones y OpenAI 4o).
4. Comparación de las consultas generadas con las consultas de referencia, tanto a nivel sintáctico como semántico (ejecución y resultados).

In [ ]:
# Conexión con la base de datos
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine, text

usuario = 'postgres'
password = 'place_rag_password'
host = 'localhost'     # o la IP/URL de tu servidor
puerto = '5432'        # puerto por defecto de PostgreSQL
base_datos = 'place_rag_db'

# Crear la URL de conexión
uri = f"postgresql+psycopg2://{usuario}:{password}@{host}:{puerto}/{base_datos}"

engine = create_engine(uri)
db = SQLDatabase.from_uri(uri)

import os
os.environ.get("HF_API_KEY")

## 1. Carga del *dataset*
A continuación, se lee el *dataset* que utilizaremos para construir y evaluar las consultas generadas por los modelos.

In [2]:
import pandas as pd

full_dataset = pd.read_csv("datasets/sampled_place_dataset_large.csv")
full_dataset = full_dataset.rename(columns={'Unnamed: 0': 'Indice'})

## 2. Definición del *system prompt*
Este *prompt* se entrega al modelo para indicarle el contexto y las reglas que debe seguir al generar las consultas SQL.

In [3]:
system_prompt = f"""
Dada una pregunta de entrada, crea una consulta de postgresql sintácticamente correcta.
Usa solo los nombres de las columnas que puedes ver en la descripción del esquema.
No consultes columnas que no existen.
Utiliza únicamente las siguientes tablas: 'entidades', 'expedientes', 'paises', 'regiones'
Esquema de la base de datos:
{db.table_info}
"""

### Función para extraer la porción de la *query* dentro del texto generado
En ocasiones, el modelo genera contenido adicional junto a la consulta. Con esta función se intenta extraer únicamente la porción de la consulta (empezando por `SELECT * ... ;`).

In [4]:
import re

def extraer_query_sql(texto):
    patron = re.compile(r"SELECT \*(?:.|\n)*?;")
    consulta = patron.findall(texto)
    return consulta if consulta else None

### Función de reordenación del *dataset*
La siguiente función `reordenar_dataframe_por_categoria` reordena el conjunto de datos en bloques con un criterio específico: cada bloque de 35 filas contiene exactamente una fila de cada categoría.

Además, se hace una división del conjunto de datos resultante en *train*, *eval* y *test*.

In [10]:
def reordenar_dataframe_por_categoria(df, col_categoria='Categoría'):
    """
    Reordena el DataFrame 'df' en bloques, de forma que cada bloque de 35 filas
    contenga exactamente una fila de cada categoría.
    """
    categorias = df[col_categoria].unique()
    
    #Comprobar que existan exactamente 35 categorías
    if len(categorias) != 35:
        raise ValueError(f"Se esperaban 35 categorías únicas, pero se encontraron {len(categorias)}.")
    
    # Agrupar filas por categoría
    grupos_por_categoria = {
        cat: g.reset_index(drop=True) 
        for cat, g in df.groupby(col_categoria)
    }
    # Comprobar que cada categoría tenga 7 filas
    for cat, subdf in grupos_por_categoria.items():
        if len(subdf) != 63:
            raise ValueError(
                f"La categoría '{cat}' no tiene exactamente 7 filas. "
                f"Encontradas: {len(subdf)}."
            ) 
    # Construir el nuevo orden de filas:
    nuevo_orden = []
    for i in range(63):
        for cat in categorias:
            # Tomamos la fila i de la categoría cat
            fila = grupos_por_categoria[cat].iloc[i]
            # Agregamos esa fila a la lista que formará el nuevo DataFrame
            nuevo_orden.append(fila)
    
    # Convertir la lista de filas en DataFrame y reindexar
    df_reordenado = pd.DataFrame(nuevo_orden).reset_index(drop=True)
    
    return df_reordenado

dataset_ordenado = reordenar_dataframe_por_categoria(full_dataset)
test_df = dataset_ordenado.iloc[2240:]   # 175 elementos

## 3. Generación de consultas con el modelo Deepseek (versión *base*)
En esta sección:
1. Cargamos el modelo `deepseek-coder-1.3b-base`.
2. Construimos la función `generar_consultas` que se encarga de crear el *prompt*, llamar al modelo y extraer la consulta generada.
3. Aplicamos dicha función sobre nuestro *test set*.

In [11]:
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer
)

model_name = "deepseek-ai/deepseek-coder-1.3b-base"
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(
    "deepseek-ai/deepseek-coder-1.3b-base",
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Función para generar consultas
La función `generar_consultas`:
1. Toma como entrada un `DataFrame` (que contiene las preguntas), el modelo y el *tokenizer*.
2. Itera sobre cada fila, formatea un *prompt* que incluye el *system_prompt* y la pregunta del *dataset*.
3. Genera la respuesta con el modelo y extrae la *query* mediante la función `extraer_query_sql`.
4. Agrega la consulta generada a una nueva columna del `DataFrame`.

In [12]:
def generar_consultas(dataset_df, model, tokenizer):
    import torch
    from tqdm import tqdm 
    consultas_generadas = []
    for index, row in tqdm(dataset_df.iterrows(), total=len(dataset_df), desc="Generando consultas"):
        success = False
        prompt_text = system_prompt + " Pregunta: " + row["Pregunta"] + " Comienza la query siempre por SELECT * y termínala siempre por ; Respuesta: SELECT *"
        inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")
        while not success:
            try:
                with torch.no_grad():
                    outputs = model.generate(**inputs, max_new_tokens=256)
                success = True
            except Exception as e:
                print(e)
        ai_msg = tokenizer.decode(outputs[0], skip_special_tokens=True)
        try:
            resultado = extraer_query_sql(ai_msg)[1]
        except:
            resultado = ai_msg
        consultas_generadas.append(resultado)
    # Añadir las listas al DataFrame como nuevas columnas
    dataset_df['Consulta Generada'] = consultas_generadas
    return dataset_df

In [13]:
test_results_df = generar_consultas(test_df, model, tokenizer)

Generando consultas: 100%|██████████| 175/175 [1:02:41<00:00, 21.49s/it]
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_8848\3443199411.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['Consulta Generada'] = consultas_generadas


In [20]:
test_results_df.to_csv('results_tests/resultados_test_deepseek_pretrain.csv', index=True)

## 4. Generación de consultas con modelo Deepseek Coder 1.3B Fine-tuned
La misma lógica se aplica, pero cargando el modelo *fine-tuneado*.

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer
)

bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model_fin_1 = AutoModelForCausalLM.from_pretrained(
    "models/deepseek-coder-ft-2025-02-02-20-12-35",
    device_map="auto",
    quantization_config=bnb_config
)
model_fin_1.config.use_cache = False

tokenizer_fin_1 = AutoTokenizer.from_pretrained("models/tokenizer-deepseek-coder-ft-2025-02-02-20-12-36")

tokenizer_fin_1.pad_token = tokenizer_fin_1.eos_token
tokenizer_fin_1.padding_side = "right"

In [16]:
test_fin_results_df = generar_consultas(test_df, model_fin_1, tokenizer_fin_1)

Generando consultas:  23%|██▎       | 40/175 [45:06<2:32:15, 67.67s/it]


KeyboardInterrupt: 

In [24]:
test_fin_results_df.to_csv("results_tests/resultados_test_deepseek_fin.csv")

## 6. Generación de consultas con OpenAI 4o
En esta sección se demuestra un ejemplo usando `ChatOpenAI` para generar las consultas, con el mismo *prompt* y el mismo conjunto de datos de prueba.

In [18]:
from langchain_openai import ChatOpenAI
os.environ.get("OPENAI_API_KEY")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
    )

In [19]:
import time
from tqdm import tqdm

from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    ChatMessage
)
consultas_generadas = []
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Generando consultas"):
    tries_left = 5
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(
            content=row["Pregunta"] + " Comienza la query siempre por SELECT * y termínala siempre por ;"
        ),
        ChatMessage(role="assistant", content="SELECT *"),
    ]
    success = False
    while not success and tries_left > 0:
        try:
            ai_msg = llm.invoke(messages)
            success = True
        except Exception as e:
            success = False
            time.sleep(3)
            tries_left -= 1
    consultas_generadas.append(ai_msg.content)

# Limpieza de algunos caracteres que el modelo tiende a introducir
openai_results_df = test_df
consultas_generadas = [consulta.replace("```sql\n", "") for consulta in consultas_generadas]
consultas_generadas = [consulta.replace("\n```", "") for consulta in consultas_generadas]
openai_results_df['Consulta Generada'] = consultas_generadas

Generando consultas: 100%|██████████| 175/175 [09:06<00:00,  3.12s/it]
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_8848\3451416933.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  openai_results_df['Consulta Generada'] = consultas_generadas


In [21]:
openai_results_df.to_csv("results_tests/resultados_test_openai.csv")

## 7. Evaluación de las consultas generadas
En esta parte se definen funciones para:
- Comparar columnas y verificar si el texto es exactamente igual.
- Normalizar consultas SQL para verificar equivalencia.
- Ejecutar las consultas en la base de datos y obtener *DataFrames* con los resultados.
- Calcular métricas como la similitud de Jaccard y el *overlap coefficient* entre conjuntos de `contract_folder_id` resultantes de dos consultas distintas.
- Verificar si las consultas son válidas sintácticamente (o si hay un error de SQL).

In [28]:
def comparar_columnas(df, col1, col2):
    """
    Compara dos columnas de un DataFrame y devuelve una lista de valores True/False
    indicando si los valores en esas columnas son exactamente iguales.
    """
    return (df[col1] == df[col2]).tolist()
    
def normalizar_sql(query):
    from sqlglot import parse_one
    """
    Normaliza una consulta SQL parseándola y generando su representación estándar.
    """
    try:
        parsed = parse_one(query)
        return parsed.sql()
    except Exception as e:
        print(f"Error al parsear la consulta: {e}")
        return None

def son_consultas_equivalentes(sql1, sql2):
    """
    Compara dos consultas SQL para determinar si son estructuralmente equivalentes.
    """
    norm_sql1 = normalizar_sql(sql1)
    norm_sql2 = normalizar_sql(sql2)
    
    if norm_sql1 is None or norm_sql2 is None:
        return False
    
    return norm_sql1.lower() == norm_sql2.lower()

def comparar_columnas_equivalentes(df, col1, col2):
    """
    Compara dos columnas de un DataFrame y devuelve una lista de valores True/False
    indicando si los valores en esas columnas son consultas SQL equivalentes.
    """
    return [son_consultas_equivalentes(sql1, sql2) for sql1, sql2 in zip(df[col1], df[col2])]

def ejecutar_query_to_df(query: str):
   # Ejecutar la consulta SQL usando SQLAlchemy para obtener columnas y datos
    with engine.connect() as connection:
        result_proxy = connection.execute(text(query))
        columns = result_proxy.keys()
        results = result_proxy.fetchall()  # Obtener todas las filas correctamente
    
    # Convertir los resultados en un DataFrame
    df = pd.DataFrame(results, columns=columns) if results else pd.DataFrame(columns=columns)
    
    return df

def comparar_resultados_consultas(df, columna_1, columna_2):
    similarity_metrics = []
    
    for index, row in df.iterrows():
        query1 = row[columna_1]
        query2 = row[columna_2]
        try: 
            df1 = ejecutar_query_to_df(query1)
            df2 = ejecutar_query_to_df(query2)
        
            if 'contract_folder_id' in df1.columns and 'contract_folder_id' in df2.columns:
                set1 = set(df1['contract_folder_id'])
                set2 = set(df2['contract_folder_id'])
                
                intersection = len(set1 & set2)
                union = len(set1 | set2)
                jaccard_similarity = intersection / union if union > 0 else 0
                overlap_coefficient = intersection / min(len(set1), len(set2)) if min(len(set1), len(set2)) > 0 else 0
                
                similarity_metrics.append({
                    'index': index,
                    'jaccard_similarity': jaccard_similarity,
                    'overlap_coefficient': overlap_coefficient
                })
        except:
            similarity_metrics.append({
                    'index': index,
                    'jaccard_similarity': "Query inválida",
                    'overlap_coefficient': "Query inválida"
                })            
    
    return pd.DataFrame(similarity_metrics)

from sqlalchemy.exc import SQLAlchemyError

def ejecutar_consultas(df, db, columna):
    resultados = []
    for idx, row in df.iterrows():
        consulta = row[columna] if pd.notna(row[columna]) else "" 
        try:
            db.run(consulta)
            resultados.append("Query válida")
        except SQLAlchemyError as e:
            error_msg = str(e.__dict__['orig']) if e.__dict__.get('orig') else str(e)
            resultados.append([idx, error_msg])
    return resultados

### Cargamos y evaluamos los resultados para cada uno de los modelos
Aquí se explica el flujo:
1. Cargamos cada CSV con los resultados generados previamente.
2. Evaluamos la validez de las *queries* (sintaxis y ejecución en la base de datos).
3. Calculamos la coincidencia exacta (`Exacta`) y la coincidencia semántica (`Semántica`).
4. Calculamos la similitud de Jaccard y el *overlap coefficient* (si se cumple la disponibilidad de la columna `contract_folder_id`).

Finalmente, se concatenan los resultados de todos los modelos para tener un `df_final_resultados`.

#### Deepseek Coder 1.3B base

In [29]:
# Deepseek Coder 1.3B base
import pandas as pd

deepseek_base_results_df = pd.read_csv("results_tests/resultados_test_deepseek_pretrain.csv")
deepseek_base_results_df["Modelo"] = "Deepseek Coder 1.3B base"

# Queries válidas
validas = ejecutar_consultas(deepseek_base_results_df, db, "Consulta Generada")
deepseek_base_results_df["Validas"] = validas

# Coincidencia exacta
deepseek_base_results_df["Exacta"] = comparar_columnas(deepseek_base_results_df, "Consulta", "Consulta Generada")

# Coincidencia semántica
deepseek_base_results_df["Semántica"] = comparar_columnas_equivalentes(deepseek_base_results_df, "Consulta", "Consulta Generada")

# Coincidencia de resultados
metricas_semejanza = comparar_resultados_consultas(deepseek_base_results_df, "Consulta", "Consulta Generada")
deepseek_base_results_df["Jaccard"] = metricas_semejanza["jaccard_similarity"]
deepseek_base_results_df["Overlap"] = metricas_semejanza["overlap_coefficient"]

Error al parsear la consulta: object of type 'float' has no len()
Error al parsear la consulta: object of type 'float' has no len()
Error al parsear la consulta: object of type 'float' has no len()


#### Deepseek Coder 1.3B fine-tuned 1

In [30]:
# Deepseek Coder 1.3B fine-tuned 1
deepseek_fin1_results_df = pd.read_csv("results_tests/resultados_test_deepseek_fin.csv")
deepseek_fin1_results_df["Modelo"] = "Deepseek Coder 1.3B fine-tuned 1"

# Queries válidas
validas = ejecutar_consultas(deepseek_fin1_results_df, db, "Consulta Generada")
deepseek_fin1_results_df["Validas"] = validas

# Coincidencia exacta
deepseek_fin1_results_df["Exacta"] = comparar_columnas(deepseek_fin1_results_df, "Consulta", "Consulta Generada")

# Coincidencia semántica
deepseek_fin1_results_df["Semántica"] = comparar_columnas_equivalentes(deepseek_fin1_results_df, "Consulta", "Consulta Generada")

# Coincidencia de resultados
metricas_semejanza = comparar_resultados_consultas(deepseek_fin1_results_df, "Consulta", "Consulta Generada")
deepseek_fin1_results_df["Jaccard"] = metricas_semejanza["jaccard_similarity"]
deepseek_fin1_results_df["Overlap"] = metricas_semejanza["overlap_coefficient"]

Error al parsear la consulta: Invalid expression / Unexpected token. Line 2, Col: 3.
  SELECT nif FROM entidades WHERE country_subentity_code = 'AT1' OR country_subentity_code = 'AT11')
 Id licitación: https://contrataciondelestado.es/wps/poc?uri=deeplink:detalle_licitacion&idEvl=Ag4n4m84


#### OpenAI 4o

In [32]:
# OpenAI 4o
openai_results_df = pd.read_csv("results_tests/resultados_test_openai.csv")
openai_results_df["Modelo"] = "OpenAI 4o"

# Queries válidas
validas = ejecutar_consultas(openai_results_df, db, "Consulta Generada")
openai_results_df["Validas"] = validas

# Coincidencia exacta
openai_results_df["Exacta"] = comparar_columnas(openai_results_df, "Consulta", "Consulta Generada")

# Coincidencia semántica
openai_results_df["Semántica"] = comparar_columnas_equivalentes(openai_results_df, "Consulta", "Consulta Generada")

# Coincidencia de resultados
metricas_semejanza = comparar_resultados_consultas(openai_results_df, "Consulta", "Consulta Generada")
openai_results_df["Jaccard"] = metricas_semejanza["jaccard_similarity"]
openai_results_df["Overlap"] = metricas_semejanza["overlap_coefficient"]

Error al parsear la consulta: Invalid expression / Unexpected token. Line 1, Col: 10.
  Lo siento, pero no puedo ayudarte con eso.
Error al parsear la consulta: Invalid expression / Unexpected token. Line 1, Col: 10.
  Lo siento, pero no puedo completar la consulta solicitada ya que no hay información suficiente en el esquema p
Error al parsear la consulta: Invalid expression / Unexpected token. Line 1, Col: 10.
  Lo siento, pero no puedo proporcionar la consulta SQL exacta que estás solicitando.
Error al parsear la consulta: Invalid expression / Unexpected token. Line 1, Col: 10.
  Lo siento, pero no puedo proporcionar la consulta solicitada ya que no hay información suficiente en el esquem


### Consolidación de resultados
Se concatenan los DataFrames con los resultados de cada modelo para poder analizarlos en conjunto.

In [33]:
# Concatenar los DataFrames y mostrar el resultado
df_final_resultados = pd.concat([
    deepseek_base_results_df,
    deepseek_fin1_results_df,
    openai_results_df
], ignore_index=True)

df_final_resultados

,Unnamed: 0,Pregunta,Consulta,Tabla,Valores,Categoría,Consulta Generada,Modelo,Validas,Exacta,Semántica,Jaccard,Overlap
0,210,Solicito información de expedientes en Cuenca.,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,{'region': 'Cuenca'},region,SELECT * FROM expedientes WHERE party_nif = 'S...,Deepseek Coder 1.3B base,"[0, no existe la columna «party_subentity_code...",False,False,Query inválida,Query inválida
1,211,Muestra los expedientes de Concesión de Servic...,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,"{'tipo': 'Concesión de Servicios', 'entidad': ...",entidad_tipo_estado_cuantia_inferior,SELECT * FROM expedientes WHERE contract_folde...,Deepseek Coder 1.3B base,Query válida,False,False,0.0,0
2,212,Solicito información de expedientes en Consejo...,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,{'entidad': 'Consejo de Administración de la S...,entidad_cuantia_superior,SELECT * FROM expedientes WHERE contract_folde...,Deepseek Coder 1.3B base,Query válida,False,False,0.0,0
3,213,Muestra los expedientes de Obras en Televisión...,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,"{'tipo': 'Obras', 'entidad': 'Televisión Públi...",entidad_tipo_estado,SELECT * FROM expedientes WHERE contract_folde...,Deepseek Coder 1.3B base,Query válida,False,False,0.0,0
4,214,Muestra los expedientes de Gestión de Servicio...,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,"{'tipo': 'Gestión de Servicios Públicos', 'ent...",entidad_tipo_cuantia_inferior,SELECT * FROM expedientes WHERE party_nif = 'S...,Deepseek Coder 1.3B base,Query válida,False,False,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,240,Muestra los expedientes de Suministros en Cent...,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,"{'tipo': 'Suministros', 'region': 'Centro (ES)...",region_tipo_cuantia_superior,SELECT * \nFROM expedientes e\nJOIN entidades ...,OpenAI 4o,Query válida,False,False,0.0,0
136,241,Muestra los expedientes Anuncio Previo de Sori...,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,"{'estado': 'Anuncio Previo', 'region': 'Soria'...",region_estado_cuantia_inferior,SELECT * \nFROM expedientes e\nJOIN entidades ...,OpenAI 4o,Query válida,False,False,0,0
137,242,Muestra los expedientes Adjudicado de Girona c...,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,"{'estado': 'Adjudicado', 'region': 'Girona', '...",region_estado_cuantia_superior,SELECT * \nFROM expedientes e\nJOIN entidades ...,OpenAI 4o,Query válida,False,False,0,0
138,243,Muestra los expedientes de Concesión de Servic...,SELECT * FROM expedientes JOIN entidades ON ex...,expedientes,"{'tipo': 'Concesión de Servicios', 'region': '...",region_tipo_estado_cuantia_inferior,SELECT *\nFROM expedientes e\nJOIN entidades e...,OpenAI 4o,Query válida,False,False,0,0


In [35]:
df_final_resultados.to_csv("results_tests/resultados_test_consolidados.csv")